In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append("../")

In [2]:
import pickle
from functools import partial, reduce
from itertools import chain, tee, islice
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from flair.embeddings import BertEmbeddings, Sentence

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
import local_config
from MWE2019.materials import NGram4List, MoeIdioms
from MWE2019.corpus import CorpusFactory
from MWE2019.corpus_index import CorpusIndex
from MWE2019.cwn_node_vec import CwnNodeVec
from MWE2019.utils import get_cache_path, install_data_cache

In [4]:
bert = BertEmbeddings('bert-base-chinese', layers='-1,-2,-3,-4', pooling_operation='mean')

In [5]:
nv_eq = CwnNodeVec(name='structeq', dimensions=100, walk_length=10, num_walks=20, p=0.5, q=2)

load CwnNodeVec from cache:  ..\MWE2019\..\data\cache_cwn_node_vec\cwn_node_vec_structeq.pkl


In [6]:
len([x for x in nv_eq.stoi.keys() if len(x) == 1])

3831

In [7]:
cwn_chars = [x for x in nv_eq.stoi.keys() if len(x) == 1]

## Build character sense vectors

In [8]:
sys.path.append("../../GWA2019")
from GWA2019 import cwn_sense_utils as utils
with open("../resources/sense_embed.pkl", "rb") as fin:
    sense_embed = pickle.load(fin)

In [9]:
next(iter(sense_embed.keys()))

'03000401'

In [10]:
from CwnGraph import CwnBase

In [11]:
cwn = CwnBase()

In [12]:
from GWA2019.cwn_sense_utils import find_examples, compute_sense_embedding

In [13]:
import torch
torch.cuda.is_available()

False

In [14]:
charac_sense_vectors = {}

In [15]:
def get_sense_vector_from_cache(sense_ids, sense_cache):
    sense_map = {}
    for sense_id_x in sense_ids:
        emb = sense_cache[sense_id_x][sense_id_x].embeddings
        sense_map[sense_id_x] = emb.mean(0)
    return sense_map

def get_char_sense_vectors(charac, cwn_inst, bert_inst, sense_cache):
    lemmas = cwn.find_lemma(f'^{ch}$')
    senses = chain.from_iterable(x.senses for x in lemmas)
    sense_ids = [x.id for x in senses]
    has_cache = all([sid_x in sense_cache for sid_x in sense_ids])
    
    if has_cache:
        sense_map = get_sense_vector_from_cache(sense_ids, sense_cache)
    else:
        sense_ids, sense_emb = compute_sense_embedding(charac, cwn_inst, bert_inst)
        sense_map = dict(zip(sense_ids, sense_emb))
    return sense_map


In [ ]:
char_senses = {}
for ch in tqdm(cwn_chars):
    try:
        char_senses[ch] = get_char_sense_vectors(ch, cwn, bert, sense_embed)
    except Exception as ex:
        print(ex)        

No valid embeddings from examples
expected a non-empty list of Tensors


In [55]:
char_senses

{'一': {'05224201': tensor([-0.0186,  0.1967, -0.2883,  ...,  0.0495,  0.2572, -1.4066]),
  '05224202': tensor([ 0.1864, -0.4059, -0.3544,  ...,  0.7837, -0.1061,  0.0499]),
  '06727201': tensor([ 0.4360,  0.1061, -0.6604,  ...,  0.8937,  0.3118, -0.6445]),
  '06727301': tensor([ 0.0879,  0.2555, -0.3506,  ...,  0.3521,  0.0166, -0.9185]),
  '06727302': tensor([ 0.3703,  0.2827, -0.0248,  ...,  0.0032,  0.1396, -0.5456]),
  '06727303': tensor([ 0.3537, -0.2587, -0.5117,  ...,  0.8127,  0.7034, -0.2929]),
  '06727304': tensor([-0.0654,  0.3799, -0.3318,  ...,  0.6671,  0.0063,  0.5669]),
  '06727305': tensor([ 0.6589, -0.0805, -1.2556,  ...,  1.1982, -0.1686, -0.0408]),
  '06727306': tensor([-0.0919,  0.4512, -0.8062,  ...,  0.4372,  0.0636,  0.2234]),
  '06727401': tensor([ 0.3337, -0.2030, -0.1241,  ...,  0.1443,  0.0999, -0.0269])}}